In [8]:
import pandas as pd

df_points = pd.read_csv('../data/final_nosplit_df_points_18_21_class.csv')

In [9]:
# add hex enc
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df_points['hex_enc']= label_encoder.fit_transform(df_points['hex_id'])

In [10]:
def on_station(row):
    if row['end_station_number'] != 0:
        return 1
    else: return 0

df_points['on_station'] = df_points.apply(on_station,axis=1)

In [11]:
zone_encoder = preprocessing.LabelEncoder()
df_points['zone_name_enc']= zone_encoder.fit_transform(df_points['zone_name'])

In [56]:
#add classification target
def f(row):
    if row['idle_time'] < 1440:
        val = '1day'
    elif row['idle_time'] < 2880:
        val = '2day'
    elif row['idle_time'] < 10080:
        val = '1week'
    else:
        val = 'longer'
    return val

df_points['idle_time_class'] = df_points.apply(f, axis=1)

In [57]:
df_points = df_points.reindex(columns=['bike_id', 'lat', 'lng', 'dt_start', 'dt_end', 'hex_id','hex_enc', 'in_zone', 'zone_name','zone_name_enc','end_station_number','on_station','temp', 'rain', 'snow', 'wind_speed', 'humidity', 'year','month', 'day', 'start_min', 'over_night', 'idle_time_next_day', 'idle_time', 'idle_time_class'])

df_points = df_points.sort_values(by=['dt_start'])
df_points

,bike_id,lat,lng,dt_start,dt_end,hex_id,hex_enc,in_zone,zone_name,zone_name_enc,...,wind_speed,humidity,year,month,day,start_min,over_night,idle_time_next_day,idle_time,idle_time_class
0,3694,51.323619,12.335887,1514764029,1,881f1a8d91fffff,320,1,Pinke Zone Leipzig West 1€,4,...,7.20,76.0,2017,12,6,1427,NaN,NaN,834,1day
1,2508,51.323619,12.335887,1514764035,1,881f1a8d91fffff,320,1,Pinke Zone Leipzig West 1€,4,...,7.20,76.0,2017,12,6,1427,NaN,NaN,2331,2day
2,41851,51.332415,12.395158,1514764638,1,881f1a1659fffff,133,1,Pinke Zone Leipzig Ost 1€,3,...,7.20,76.0,2017,12,6,1437,NaN,NaN,1525,2day
3,1859,51.357471,12.377097,1514765836,1,881f1a8c83fffff,257,1,Blaue Zone Leipzig Ost 0€,1,...,9.30,76.0,2018,1,0,17,NaN,NaN,26,1day
4,20171,51.363000,12.373595,1514767793,1,881f1a8c83fffff,257,1,Pinke Zone Leipzig Ost 1€,3,...,9.30,76.0,2018,1,0,50,NaN,NaN,3549,1week
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781370,72058,51.317569,12.382730,1640897593,1,881f1a1609fffff,98,1,Blaue Zone Leipzig Ost 0€,1,...,8.94,83.0,2021,12,3,1253,NaN,NaN,112,1day
1781371,71949,51.334711,12.356763,1640897835,1,881f1a1649fffff,126,1,Pinke Zone Leipzig Ost 1€,3,...,8.94,83.0,2021,12,3,1257,NaN,NaN,25,1day
1781372,97785,51.367276,12.369792,1640898086,1,881f1a8c8bfffff,261,1,Blaue Zone Leipzig Ost 0€,1,...,8.05,84.0,2021,12,3,1261,NaN,NaN,48,1day
1781373,41738,51.366942,12.370322,1640900918,1,881f1a8c8bfffff,261,1,Pinke Zone Leipzig Ost 1€,3,...,8.05,84.0,2021,12,3,1309,NaN,NaN,43,1day


In [58]:
#df_points.to_csv('../data/final_nosplit_df_points_18_21_class.csv', index=False)

In [59]:
df_points.value_counts(['idle_time_class'])

idle_time_class
1day               1669080
2day                 69195
1week                39102
longer                3998
dtype: int64

In [60]:
df_points.value_counts(['idle_time_class'])

idle_time_class
1day               1669080
2day                 69195
1week                39102
longer                3998
dtype: int64

# test

In [69]:
df_points = df_points[(df_points['lat'] != 0)]

In [75]:
df_points = df_points[(df_points['idle_time'] > 10)]

In [71]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import wandb

#idle_time_data = pd.read_csv('../data/final_nosplit_df_points_18_21_class.csv')

idle_time_data = df_points

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'wind_speed', 'humidity', 'dt_start',
              'hex_enc', 'start_min', 'year', 'month', 'day', 'on_station', 'in_zone', 'zone_name_enc']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)


In [72]:
import numpy as np

np.unique(y_test, return_counts=True)

(array(['1day', '1week', '2day', 'longer'], dtype=object),
 array([314113,   4506,   9032,    367]))

In [73]:
model = RandomForestClassifier(n_jobs=2,n_estimators=128)

In [74]:
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score


def eval_classification(y_test,y_pred,labels):
    # Metrics
    # Accuracy, precision, recall
    acc = accuracy_score(y_test, y_pred.ravel())
    macro_precision = precision_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=labels)
    micro_precision = precision_score(y_test.ravel(), y_pred.ravel(), average='micro', labels=labels)
    macro_recall = recall_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=labels)
    micro_recall = recall_score(y_test.ravel(), y_pred.ravel(), average='micro', labels=labels)

    macro_f1 = f1_score(y_test.ravel(), y_pred.ravel(), average='macro', labels=labels)
    micro_f1 = f1_score(y_test.ravel(), y_pred.ravel(), average='micro', labels=labels)

    print(acc)
    print(macro_precision, micro_precision)
    print(macro_recall,micro_recall)
    print(macro_f1,micro_f1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test,y_pred)

eval_classification(y_test,y_pred,[1,2,3,4])

print(classification_report(y_test,y_pred))